In [28]:
#Library imports
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import csv
import pyvisa
from copy import deepcopy

#Options
vis = False #Variable to set whether the pyvisa stuff is run or not
flTemp = True #Set whether the floating temperature display will be on or not
pix_read_only = True #Set whether the temp values for only the pixels specified will be evaluated

#Initialise important variables
dev = -1 #Device ID of the camera - 0 if it is the only camera device
pixels = [] #List of the coordinates of points to monitor
data = [] #This variable will hold the data for every frame
D = []

#PyVisa Initialisation of communication with Keithley K-488B
if vis:
    rm = pyvisa.ResourceManager() #Creates resource manager object
    #rm.list_resources() #List the resources that are available to the computer
    inst = rm.open_resource('GPIB0::15::INSTR')
    #print(inst.query("*IDN?"))
    #P=inst.query("Out2.Value?")

    cmdString="Out2.PID.Ramp"
    setValue=0.033
    inst.write(cmdString + ' ' + str(setValue))

    cmdString="Out2.PID.setpoint"
    setValue=310
    inst.write(cmdString + ' ' + str(setValue))

#Camera Input General Settings
width = 256 #Width of the sensor input
height = 192 #Height of the sensor input
scale = 3 #Value of the scale multiplier for interpolation
newWidth = scale*width
newHeight = scale*height
alpha = 1.0 #Contrast control variable
colormap = 0 #ID of the colormap that is to be applied to the image
font = cv2.FONT_HERSHEY_SIMPLEX #Font of the window
dispFullscreen = False #Whether the fullscreen mode is on or not
rad = 0 #Blur Radius value
threshold = 2
hud = True
recording = False
elapsed = "00:00:00"
snaptime = "None"
fi = './' #Path of the directory where the data will be saved
dataRec = False #Whether the data collection mode is on or off

#Init Camera
cap = cv2.VideoCapture(dev,cv2.CAP_V4L)
cap.set(cv2.CAP_PROP_CONVERT_RGB, 0.0) #Prevents the capture device from converting raw data to RGB
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('Y','1','6',' '))
cap.set(cv2.CAP_PROP_FORMAT, -1)

#Window init settings
cv2.namedWindow('ThermalCam', cv2.WINDOW_GUI_NORMAL)
cv2.resizeWindow('ThermalCam', newWidth, newHeight)

#Functions
def rec():
    global fi
    now = time.strftime("%Y%m%d--%H%M%S")
    #Do not use mp4
    videoOut = cv2.VideoWriter(fi+now+'output.avi', cv2.VideoWriter_fourcc(*'XVID'),25, (newWidth,newHeight))
    return(videoOut)

def snapshot(heatmap):
    global fi
    now = time.strftime("%Y%m%d-%H%M%S")
    snaptime = time.strftime("%H:%M:%S")
    cv2.imwrite(fi+"TC001"+now+".png", heatmap)
    return snaptime

def getTemp(tdata,coor):
    hi = tdata[coor[0]][coor[1]][0]
    lo = tdata[coor[0]][coor[1]][1]
    lo = lo*256
    rawtemp = hi+lo
    temp = round((rawtemp/64),2)
    #print('Coor ->'+str(coor)+' T>'+str(tempc))
    #Returns temperature in Kelvin, rounded off the to second digit
    return temp

def drawData(coor,temp):
    global heatmap # It is the image that the data will be drawn on
    global scale # Scale of the interpolated image so that the coordinates can be shifted
    x = coor[0]*scale
    y = coor[1]*scale
    
    # draw crosshairs
    cv2.line(heatmap,(x,y+20),\
    (x,y-20),(255,255,255),2) #vline
    cv2.line(heatmap,(x+20,y),\
    (x-20,y),(255,255,255),2) #hline

    cv2.line(heatmap,(x,y+20),\
    (x,y-20),(0,0,0),1) #vline
    cv2.line(heatmap,(x+20,y),\
    (x-20,y),(0,0,0),1) #hline
    #show temp
    cv2.putText(heatmap,str(temp)+' K', (x+10, y-10),\
    cv2.FONT_HERSHEY_SIMPLEX, 0.45,(0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(heatmap,str(temp)+' K', (x+10, y-10),\
    cv2.FONT_HERSHEY_SIMPLEX, 0.45,(0, 255, 255), 1, cv2.LINE_AA)
    return True

#Set up for the clicking top select pixel functionality
def click_event(event, x, y, flags, params):
    global pixels
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f'({x},{y})')
        pixels.append([x,y])

cv2.setMouseCallback('ThermalCam', click_event)

start = time.time()

while cap.isOpened():
    # Get a frame from the camera
    ret, frame = cap.read()
    # Get data from PID
    if vis:
        T = inst.query("In2.Value?")
    #time.sleep(1)
    #Begin Processing of the image
    if ret:
        s = time.time() #Timestamp for this frame capture
        imdata, th = np.array_split(frame,2) #Splits the array into temp data part and img part
        t_frame = [] 
        # Get the temperature values of the pixels on the screen
        for i in range(len(th)):
            row = []
            for j in range(len(th[0])):
                #print([i,j])
                if pix_read_only:
                    if [i,j] not in pixels:
                        continue
                    else:
                        pass
                temp = round((th[i][j][1]*256 + th[i][j][0])/64,2)
                row.append(temp)
            t_frame.append(list(row))
        if vis:
            data = [abs(start-s),deepcopy(t_frame),T]
        else:
            data = [abs(start-s),deepcopy(t_frame)]
        
        # Convert the real image to RGB
        bgr = cv2.cvtColor(imdata,  cv2.COLOR_YUV2BGR_YUYV)
        #Contrast
        bgr = cv2.convertScaleAbs(bgr, alpha=alpha)#Contrast
        #bicubic interpolate, upscale and blur
        bgr = cv2.resize(bgr,(newWidth,newHeight),interpolation=cv2.INTER_CUBIC)#Scale up!
        if rad>0:
            bgr = cv2.blur(bgr,(rad,rad))

        #apply colormap
        if colormap == 0:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_JET)
            cmapText = 'Jet'
        if colormap == 1:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_HOT)
            cmapText = 'Hot'
        if colormap == 2:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_MAGMA)
            cmapText = 'Magma'
        if colormap == 3:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_INFERNO)
            cmapText = 'Inferno'
        if colormap == 4:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_PLASMA)
            cmapText = 'Plasma'
        if colormap == 5:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_BONE)
            cmapText = 'Bone'
        if colormap == 6:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_SPRING)
            cmapText = 'Spring'
        if colormap == 7:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_AUTUMN)
            cmapText = 'Autumn'
        if colormap == 8:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_VIRIDIS)
            cmapText = 'Viridis'
        if colormap == 9:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_PARULA)
            cmapText = 'Parula'
        if colormap == 10:
            heatmap = cv2.applyColorMap(bgr, cv2.COLORMAP_RAINBOW)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
            cmapText = 'Inv Rainbow'
        #Store the Data for this frame
        D.append(deepcopy(data))
        
        #Display the frame on the window
        cv2.imshow('ThermalCam',heatmap)
        
        keyPress = cv2.waitKey(1)
        
        if keyPress == ord('m'): #m to cycle through color maps
            colormap += 1
            if colormap == 11:
                colormap = 0
        if keyPress == ord('q'): #q to QUIT the application
            cap.release()
            cv2.destroyAllWindows()
            break

In [29]:
cv2.destroyAllWindows()

In [3]:
len(D)

0

In [28]:
D[20][1][92][188]

295.56

In [3]:
pip install opencv-python --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------